In [158]:
# try:
#     get_ipython().system('pip install pandas_datareader')
#     get_ipython().system('pip install pandas')
#     get_ipython().system('pip install matplotlib')
#     get_ipython().system('pip install requests')
#     get_ipython().system('pip install sympy')
#     get_ipython().system('pip install scipy')
#     get_ipython().system('pip install numpy')
#     get_ipython().system('pip install beautifulsoup4')
# except Exception:
#     pass

In [159]:
import os
import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import pandas as pd
import requests
from sympy.solvers import solve
from sympy import Symbol
from scipy.optimize import fsolve
import numpy as np
from bs4 import BeautifulSoup
import threading

In [171]:
tickers = []

with open('tickers.csv') as fh:
    for line in fh:
        row = line.strip()
        tickers.append(row)
print(tickers)

['nyse:WGO', 'nyse:WWW']


In [164]:
def rdpModelReturn(payoutRatio, PE, returnOnEquity):
    return (1 + returnOnEquity*(1-payoutRatio))*(1+payoutRatio/PE)-1

In [165]:
# cookies cannot be int only string

cookies = {
    "_gid": "GA1.2.1666171758.1680587936",
    "gclid": "undefined",
    "hunt7586-_zldp": "ss+YrC93NjPqF7vKLbOAPFEzKpujuf37zXoWPDOEmLWmo3KEoDowjfvVgOHufVhm7uRfjfIQJjw=",
    "hunt7586-_zldt": "68aa0a2f-eeef-4564-81a5-65e3099fd9b9-1",
    "ajs_user_id": "60d8645aa921b600176fc31e",
    "JDCID": "dc89cc652973e72ed3b483303a4bce5fa991b812a29997522c7401a8782b6cad.5439c914472955e6ecc4ac72cfa2a8c48dfe5358c8de3df5e4b63278cc6e5439705672157f5bbd49210a4b2cc4dec3851703061b2613e31a45162e61f84510a469b436eb405ba50db920a0f8f433b0620c295848890c",
    "_ga": "GA1.1.467613354.1680587936",
    "_ga_BT0RBD348J": "GS1.1.1680586104.7.1.1680587950.44.0.0",
 
    "ajs_anonymous_id": "944bd501-bbbc-464a-a2bb-1baaba009b9d",
    "jitta_nu": "2"
}

In [166]:
def convert_row_to_data(row):
    r = row.find_all('span')
    c = map(lambda x: p2f(x.text), r)
    return list(c)

def p2f(x):
    cx = x.replace('%', '').replace(',', '')
    print(x)

    return None if cx == '- -' or cx == '?' else float(cx)/100

In [167]:
def calculate(ticker):
    print(ticker)
    with requests.post('https://www.jitta.com/stock/' + ticker + '/factsheet', cookies=cookies) as r:
        soup = BeautifulSoup(r.content, 'lxml')
        row = soup.find_all('div', {'class': 'dAGwCX'})
        if len(row) == 0:
            return  {
            "TICKER": ticker,
            "AVG_G_THEORY": None,
            "AVG_ROE": None,
            "AVG_DPR": None,
            "LAST_G_THEORY": None,
            "LAST_ROE": None,
            "LAST_DPR": None,
            "NEGATIVE_ROE": None,
            "COUNT_ROE": None,
        }

        EPS = convert_row_to_data(row[1]) 
        print(EPS)
        ROE = convert_row_to_data(row[20])
        ROA = convert_row_to_data(row[21])
        DE = convert_row_to_data(row[23])
        DPR = convert_row_to_data(row[24])
        avg_ROE = 0
        last_ROE = 0
        negative_ROE = 0
        avg_DPR = 0
        last_DPR = 0
        c_ROE = 0
        avg_g_theory = 0
        last_g_theory = 0
        avg_g_empirical = 0
        c_g = 0
        print(ROE)
        for i in range(0, len(ROE)):
            if ROE[i] == None:
                continue

            dpr = 0 if DPR[i] == None else DPR[i]
            avg_g_theory += ROE[i] * (1 - dpr)
            avg_ROE += ROE[i]
            avg_DPR += dpr
            c_ROE += 1
            last_ROE = ROE[i]
            last_DPR = dpr
            last_g_theory = ROE[i] * (1 - dpr)

            if ROE[i] < 0:
                negative_ROE += 1

        avg_ROE /= c_ROE
        avg_DPR /= c_ROE
        avg_DPR = 0 if avg_DPR == None else avg_DPR
        avg_g_theory /= c_ROE

        return {
            "TICKER": ticker,
            "AVG_G_THEORY": avg_g_theory,
            "AVG_ROE": avg_ROE,
            "AVG_DPR": avg_DPR,
            "LAST_G_THEORY": last_g_theory,
            "LAST_ROE": last_ROE,
            "LAST_DPR": last_DPR,
            "NEGATIVE_ROE": negative_ROE,
            "COUNT_ROE": c_ROE,
        }

In [168]:
f = open('stock-screen.csv', 'w')
f.write('ticker|AVG_G_THEORY|AVG_ROE|AVG_DPR|LAST_G_THEORY|LAST_ROE|LAST_DPR|NEGATIVE_ROE|COUNT_ROE\n')

def fetch_stock(ticker):
    data = calculate(ticker)
    print(data)
    if data['AVG_G_THEORY'] == None:
        return
        
    f.write(ticker + '|' + 
        '{:.4f}'.format(data['AVG_G_THEORY']) + '|' +
        '{:.4f}'.format(data['AVG_ROE']) + '|' +
        '{:.4f}'.format(data['AVG_DPR']) + '|' +
        '{:.4f}'.format(data['LAST_G_THEORY']) + '|' +
        '{:.4f}'.format(data['LAST_ROE']) + '|' +
        '{:.4f}'.format(data['LAST_DPR']) + '|' + 
        '{}'.format(data['NEGATIVE_ROE']) + '|' + 
        '{}'.format(data['COUNT_ROE']) + '\n'
       )


In [169]:
threads  = []
for ticker in tickers:
    fetch_stock(ticker)
#      t = threading.Thread(target=fetch_stock, args=(ticker,))
#      threads.append(t)
#      t.start()
        
# for t in threads:
#     t.join()   

f.close()

nyse:WGO
1.64
1.52
1.68
2.32
3.22
3.52
1.84
8.28
11.8
- -
?
[0.016399999999999998, 0.0152, 0.0168, 0.0232, 0.0322, 0.0352, 0.0184, 0.0828, 0.11800000000000001, None, None]
24.8%
19.9%
18.6%
20.1%
21.0%
19.2%
8.4%
29.9%
33.7%
- -
?
13.5%
11.4%
12.1%
11.0%
10.5%
10.4%
4.4%
14.9%
17.4%
- -
?
0.86
0.64
0.46
1.04
0.97
0.75
1.07
0.95
0.91
- -
?
- -
23.7%
23.8%
17.2%
12.4%
12.2%
23.9%
5.8%
6.1%
- -
?
[0.248, 0.19899999999999998, 0.18600000000000003, 0.201, 0.21, 0.192, 0.084, 0.299, 0.337, None, None]
{'TICKER': 'nyse:WGO', 'AVG_G_THEORY': 0.19139533333333333, 'AVG_ROE': 0.21733333333333332, 'AVG_DPR': 0.139, 'LAST_G_THEORY': 0.31644300000000003, 'LAST_ROE': 0.337, 'LAST_DPR': 0.061, 'NEGATIVE_ROE': 0, 'COUNT_ROE': 9}
